# 0. Setup paths

#### Setting up paths before working makes it a lot simpler to write the code

In [54]:
WORKSPACE_PATH = "tensorflow/workspace"
SCRIPTS_PATH = "tensorflow/scripts"
API_MODEL_PATH = "tensorflow/models"
ANNOTATION_PATH = "tensorflow/workspace/annotations"
IMAGE_PATH ="tensorflow/workspace/images"
MODEL_PATH = "tensorflow/workspace/models"
PRETRAINED_MODEL_PATH = "tensorflow/workspace/pre_trained_model"
CONFIG_PATH = MODEL_PATH + '/' + CUSTOM_MODEL_NAME + '/pipeline.config'
CHECKPOINT_PATH = "tensorflow/workspace/models/my_ssd_mobnet"

#notice the difference between / and \

# 1. Create label map

#### Labels of all of the labels. In this case mask and no_mask. It will be in protobuf.


In [2]:
labels = [{'name':'mask','id':1},{'name':'no_mask','id':2}]


#### Output this dictionary into a label map. Was having problem with the code, so I just manually wrote the protobuf file

In [17]:

with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f: #save the mapping into the annotations folder
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF Record

#### Generate TFRecord using the script from the official Tensorflow Object Detection tutorial

In [20]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: tensorflow/workspace/annotations/train.record


2023-01-06 12:05:19.516497: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


Successfully created the TFRecord file: tensorflow/workspace/annotations/test.record


2023-01-06 12:05:36.200980: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


# 3. Download pre-trained models

#### I just did it manually from https://github.com/tensorflow/models

# 4. Copy the model config to training folder

In [23]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

!mkdir {'tensorflow\workspace\models\\'+ CUSTOM_MODEL_NAME} #make the directory
!copy {PRETRAINED_MODEL_PATH+ '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/' + CUSTOM_MODEL_NAME} #copy the file 

A subdirectory or file tensorflow\workspace\models\my_ssd_mobnet already exists.
Error occurred while processing: tensorflow\workspace\models\my_ssd_mobnet.


The syntax of the command is incorrect.


# 5. Update config for transfer learning

In [24]:
import tensorflow as tf
from object_detection.utils import config_util # for grabbing the config file
from object_detection.protos import pipeline_pb2 # for editing the config file
from google.protobuf import text_format #

In [26]:
CONFIG_PATH = MODEL_PATH + '/' + CUSTOM_MODEL_NAME + '/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [31]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig() #create a template config
with tf.io.gfile.GFile(CONFIG_PATH, 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [42]:
pipeline_config.model.ssd.num_classes = 2 # number of labels, (in this case mask and no mask)
pipeline_config.train_config.batch_size = 4 #higher the hardware capability, higher trhe batch size can be
pipeline_config.train_config.fine_tune_checkpoint = 'tensorflow/workspace/pre_trained_model/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'#where we want to start training from 
pipeline_config.train_config.fine_tune_checkpoint_type = 'detection' # parameter
pipeline_config.train_input_reader.label_map_path = ANNOTATION_PATH + '/label_map.pbtxt' # label map path
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record'] #specify train record path
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt' #for the testing
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record'] # path for test record

In [43]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:
    f.write(config_text)

# 6. Train the model 

In [44]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(API_MODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


#### Copy this command and run this in command prompt

# 7. Load train model from checkpoint 

In [46]:
import os
from object_detection.utils import label_map_util #work with label map
from object_detection.utils import visualization_utils as viz_utils #visualize the box on the frame
from object_detection.builders import model_builder #build the model from checkpoint

In [55]:
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH) #Grab the modified pipeline
detection_model = model_builder.build(model_config=configs['model'], is_training = False) #Build the model using model config

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-11')).expect_partial() #Load the latest checkpoint (ckpt-32.index) into the pipeline

#### Write the detection function

In [68]:
@tf.function #Tensorflow function
def detect_fn(image): #passing through image argument
    image, shapes = detection_model.preprocess(image) #resize the image
    prediction_dict = detection_model.predict(image, shapes) #predict
    detections = detection_model.postprocess(prediction_dict, shapes) #postprocess
    return detections

# 8.  Test realtime

In [56]:
import cv2
import numpy as np

In [59]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+ '/label_map.pbtxt')

In [60]:
category_index

{1: {'id': 1, 'name': 'mask'}, 2: {'id': 2, 'name': 'no_mask'}}

In [61]:
cap = cv2.VideoCapture(0) #Access webcam
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [69]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

In [ ]:
detections = detect_fn(input_tensor)